In [11]:
import csv
import psycopg2
from psycopg2 import sql
import chardet

def detect_file_encoding(csv_file):
    """Detecta a codificação de um arquivo CSV usando chardet."""
    with open(csv_file, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(100000))  # Lê os primeiros 100KB
    return result['encoding']

def create_table_from_csv(csv_file, table_name, db_config):
    """Cria uma tabela no PostgreSQL a partir de um arquivo CSV."""
    # Detecta a codificação do arquivo CSV
    encoding = detect_file_encoding(csv_file)
    print(f"Codificação detectada: {encoding}")
    
    # Lê o arquivo CSV com a codificação detectada
    with open(csv_file, 'r', encoding=encoding, errors='replace') as file:
        csv_reader = csv.reader(file, delimiter=';')
        headers = next(csv_reader)
    
    # Mapeia tipos de dados
    type_mapping = {
        'ChEMBL ID': 'TEXT',
        'Journal': 'TEXT',
        'Source ID': 'INTEGER',
        'Source Description': 'TEXT',
        'Title': 'TEXT',
        'PubMed ID': 'TEXT',
        'DOI': 'TEXT',
        'Patent ID': 'TEXT',
        'Authors': 'TEXT',
        'Year': 'INTEGER',
        'Volume': 'TEXT',
        'Issue': 'TEXT',
        'First Page': 'TEXT',
        'Last Page': 'TEXT',
        'Document Type': 'TEXT',
        'Abstract': 'TEXT',
        'Compounds': 'INTEGER',
        'Targets': 'INTEGER',
        'Activities': 'INTEGER',
        'ChEMBL Release': 'TEXT',
        'ChEMBL Release Creation Date': 'DATE'
    }

    # Gera comando SQL para criar a tabela
    create_table_query = sql.SQL("CREATE TABLE IF NOT EXISTS {} (\n").format(sql.Identifier(table_name))
    columns = []
    for header in headers:
        column_type = type_mapping.get(header, 'TEXT')  # Usa TEXT como tipo padrão
        columns.append(sql.SQL("{} {}").format(sql.Identifier(header), sql.SQL(column_type)))
    create_table_query += sql.SQL(",\n").join(columns)
    create_table_query += sql.SQL("\n);")
    
    # Conecta ao PostgreSQL e cria a tabela
    conn = None
    cur = None
    try:
        print("Conectando ao banco de dados PostgreSQL...")
        conn = psycopg2.connect(**db_config)
        cur = conn.cursor()
        print("Executando comando SQL para criar a tabela...")
        cur.execute(create_table_query)
        conn.commit()
        print(f"Tabela {table_name} criada com sucesso!")

        # Prepara a inserção dos dados
        insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier(table_name),
            sql.SQL(', ').join(map(sql.Identifier, headers)),
            sql.SQL(', ').join(sql.Placeholder() * len(headers))
        )

        # Insere cada linha do CSV
        for row in csv_reader:
            cur.execute(insert_query, row)
        
        conn.commit()
        print(f"Dados inseridos com sucesso na tabela {table_name}!")

    except (Exception, psycopg2.Error) as error:
        print("Erro ao conectar ao PostgreSQL ou criar a tabela:", error)
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()
        print("Conexão com o PostgreSQL fechada.")

# Configuração do banco de dados
db_config = {
    "dbname": "testando",
    "user": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "54321"
}

# Uso do script
csv_file = 'chagas.csv'
table_name = 'artigos_cientificos'

try:
    create_table_from_csv(csv_file, table_name, db_config)
except Exception as e:
    print(f"Ocorreu um erro ao executar o script: {e}")


Codificação detectada: utf-8
Conectando ao banco de dados PostgreSQL...
Executando comando SQL para criar a tabela...
Tabela artigos_cientificos criada com sucesso!
Erro ao conectar ao PostgreSQL ou criar a tabela: I/O operation on closed file.
Conexão com o PostgreSQL fechada.
